In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1")


In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "France"})

print(prompt)

ai_message = llm.invoke(prompt_template.invoke({
    "country": "France"
}))

print(ai_message)

output_parser = StrOutputParser()

answer = output_parser.invoke(
    llm.invoke(prompt_template.invoke({
        "country": "France"
    }))
)

text='What is the capital of France? Return the name of the city only'
content='Paris' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2025-07-31T14:04:35.1315698Z', 'done': True, 'done_reason': 'stop', 'total_duration': 275066700, 'load_duration': 24187200, 'prompt_eval_count': 24, 'prompt_eval_duration': 150445100, 'eval_count': 2, 'eval_duration': 100434400, 'model_name': 'llama3.1'} id='run--a2228701-4ec0-419a-8443-ca5a6ca01649-0' usage_metadata={'input_tokens': 24, 'output_tokens': 2, 'total_tokens': 26}


In [6]:
ai_message # Base Message의 일종

AIMessage(content='Paris', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-07-31T14:04:35.1315698Z', 'done': True, 'done_reason': 'stop', 'total_duration': 275066700, 'load_duration': 24187200, 'prompt_eval_count': 24, 'prompt_eval_duration': 150445100, 'eval_count': 2, 'eval_duration': 100434400, 'model_name': 'llama3.1'}, id='run--a2228701-4ec0-419a-8443-ca5a6ca01649-0', usage_metadata={'input_tokens': 24, 'output_tokens': 2, 'total_tokens': 26})

In [7]:
answer # String

'Paris'

from langchain_core.output_parsers import StrOutputParser
- output_parser를 이용하여 사용자가 원하는 결과만 출력이 가능하다.

In [10]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template= """
    Give following information about {country}:
    - Capital
    - Population
    - Language
    - Currency

    return it in JSON format and return the JSON dictionary only
    """,
    input_variables=["country"],
)

country_detail_prompt.invoke({
    "country": "France"
})

output_parser = JsonOutputParser()

output_parser.invoke(
    llm.invoke(country_detail_prompt.invoke({"country": "France"}))
)

{'capital': 'Paris',
 'population': '67.2 million (2020 est.)',
 'language': 'French',
 'currency': 'Euro'}

JsonOutputParser가 잘 동작하지 않을 떄는 다음과 같은 클래스를 정의하여 사용한다.

from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")

In [12]:
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")

structued_llm = llm.with_structured_output(CountryDetail)

In [13]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template= """
    Give following information about {country}:
    - Capital
    - Population
    - Language
    - Currency

    return it in JSON format and return the JSON dictionary only
    """,
    input_variables=["country"],
)

country_detail_prompt.invoke({
    "country": "France"
})

json_ai_message = structued_llm.invoke(country_detail_prompt.invoke({
    "country": "France"
}))

In [15]:
json_ai_message

CountryDetail(capital='Paris', population=67264767, language='French', currency='Euro')

In [16]:
json_ai_message.model_dump()

{'capital': 'Paris',
 'population': 67264767,
 'language': 'French',
 'currency': 'Euro'}